In [15]:
import sqlite3
import pandas as pd
import os
import datetime as dt

In [4]:
def strtime_to_timestamp(value):
    year = int(value[0:4])
    month = int(value[4:6])
    day = int(value[6:8])
    hour = int(value[-8:-6])
    minute = int(value[-5:-3])
    return pd.Timestamp(year=year, month=month, day=day,hour=hour, minute=minute)


In [35]:
db = sqlite3.connect(os.getcwd()+'/ticks.db')
tickers = ["AAPL", "META"]
oldest_date = {}
latest_date = {}
for ticker in tickers:
    c= db.cursor()
    c.execute("CREATE TABLE IF NOT EXISTS TICKER_{} (time datetime primary key, price real(15,5), volume integer)".format(ticker))

    oldest_date[ticker] = strtime_to_timestamp(c.execute("SELECT time FROM TICKER_{} ORDER BY time DESC LIMIT 1".format(ticker)).fetchall()[0][0][:-7])
    latest_date[ticker] = strtime_to_timestamp(c.execute("SELECT time FROM TICKER_{} ORDER BY time ASC LIMIT 1".format(ticker)).fetchall()[0][0][:-7])

    print(dt.date.today() - dt.timedelta(days=30))
    date_oldest = latest_date[ticker].date()
    if date_oldest < dt.date.today() - dt.timedelta(days=30):
        year = date_oldest.year
        month = date_oldest.month
        day = date_oldest.day
        sql_time = f'{year}-{month}-{day}'
        c.execute('''DELETE FROM TICKER_{} WHERE time < '{}';'''.format(ticker, sql_time))
    
    elif date_oldest > dt.date.today() - dt.timedelta(days=30):
        #we will need to implement reqHistData here
        pass

    


print(oldest_date)
print(latest_date)

2022-06-18
2022-06-18
{'AAPL': Timestamp('2022-06-27 15:03:00'), 'META': Timestamp('2022-06-23 14:04:00')}
{'AAPL': Timestamp('2022-06-27 14:27:00'), 'META': Timestamp('2022-06-23 13:35:00')}


In [34]:
lol = latest_date["AAPL"].date()
lol.year

2022

In [44]:
c.execute('''delete from TICKER_AAPL where time not in
( SELECT * FROM 
    (select min(time) from TICKER_AAPL group by day(time)) AS temp_tab
);''')

OperationalError: no such function: day

In [49]:
from main import *
histData(tickers.index(ticker),usTechStk(ticker),'3600 S', '30 secs')

NameError: name 'app' is not defined